In [1]:
import sys
sys.path.append('../')
import wrangle

In [58]:
import unicodedata
import re

In [2]:
import nlp

In [3]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [6]:
df, data_dict = wrangle.wrangle_data()

In [4]:
import pyLDAvis
import pyLDAvis.sklearn

In [5]:
pyLDAvis.enable_notebook()

In [12]:
data_dict.loc['q07']

column_name                                              future_res
is_required                                                   False
from_orig_file                                                 True
in_wrangle                                                     True
char_type                                                   numeric
function                                                categorical
data_type                                                  category
encoder                                                         NaN
survey_section                                                 Work
question_group                                      Future Research
question_text     What types of research are you (or your  team)...
Name: q07, dtype: object

resp_id
284           both qualitative and quantitative research
288                                             Multiple
294                                            Online FG
281    surveys, interviews, focus groups, secondary d...
280                                                  NaN
                             ...                        
869                                          Qual, quant
879                                          diary study
136    I don't know about the whole organisation, but...
74     I would like to use geofencing techniques, mob...
2                                                    NaN
Name: future_res, Length: 726, dtype: category
Categories (576, object): [- Cultural probes\n- Live A/B testing\n- Dairy..., - Making more use of remote testing possibilit..., /, 30 second test, user interviews, ..., what do you mean by "type"?, would like to incorporate more quant research ..., would like to open up toolkit to use panel stu..., would like to up 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 78 columns):
job_title                         726 non-null category
job_taxo                          726 non-null category
job_id                            723 non-null category
job_conduct_res                   726 non-null int64
job_analyze_res                   726 non-null int64
job_buy_res_report                726 non-null int64
job_manage_res_proj               726 non-null int64
job_observe_res                   726 non-null int64
job_plan_res                      726 non-null int64
job_teach_res                     726 non-null int64
job_advocate_res                  726 non-null int64
job_hire_res_vendor               726 non-null int64
job_lead_res_team                 726 non-null int64
num_employees                     723 non-null category
num_researchers                   722 non-null category
primary_industry                  726 non-null category
types_res_used               

In [15]:
future_research = df.future_res.dropna().apply(nlp.lemmatize)

## TEST

In [49]:
def lemmatize(text):
    """
    accept some text and return the text after applying lemmatization to each word.
    Use with .apply to a Pandas Series
    """
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    return ' '.join(lemmas)

lemmatize(lemma_test)

'True user research usability studies, ethnographic research, customer interview (we don’t do any now!)'

In [54]:
def basic_clean(text):
    """
    Lowercase everything
    Normalize unicode characters
    Replace anything that is not a letter, number, whitespace or a single quote
    """
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r"[^a-z0-9'\s]", '', text)
    text = re.sub(r"[\r|\n|\r\n]+", ' ', text)
    return text

In [61]:
clean_lemma_test = basic_clean(lemma_test)

In [64]:
nlp.lemmatize(clean_lemma_test)

'true user research usability study ethnographic research customer interview we dont do any now'

In [52]:
lemmas

['True',
 'user',
 'research',
 'usability',
 'studies,',
 'ethnographic',
 'research,',
 'customer',
 'interview',
 '(we',
 'don’t',
 'do',
 'any',
 'now!)']

In [40]:
future_research[future_research.str.find('studies')>0]

resp_id
833    True user research usability studies, ethnogra...
783    Diary studies, more contextual, increase in qu...
791    All the method out there have a time and place...
697                Unmoderated usability studies, survey
754    More field studies, but my team more training ...
854    More contextual/observational studies, unmoder...
522    Just about everything. Still figuring out the ...
476    Love/Breakup Letters, Diary studies, relations...
486    Benchmark, true intent studies, more working w...
856    We are considering exploring activity like dia...
467    Longitudinal research and diary studies, affec...
514                           Diary studies, card sorts.
867                               Lots of field studies.
395                Diary studies, competitive evaluation
848     More quantitative surveys. Longitudinal studies.
348                         Diary studies, on site visit
390    Surveys, live/remote/in-home interviews, diary...
273    More observation

In [42]:
lemma_test = future_research.loc[833]

In [44]:
type(lemma_test)

str

In [45]:
nlp.lemmatize(lemma_test)

'True user research usability studies, ethnographic research, customer interview (we don’t do any now!)'

## TEST

In [8]:
df.recommendations.dropna().apply(nlp.lemmatize)

resp_id
284                         research-practice connection
288    Don’t create schedule conflict with other rele...
294                                     Looking forward!
295    I actually feel like the biggest contribution ...
290                Go ahead! Your prestige is a guaranty
                             ...                        
853    Research in agile, research on a shoestring bu...
869    provide tool and resource and support for netw...
879          Make it affordable. Somewhere in Europe. :)
136    My goal is to provide constructive criticism, ...
74     Instead of trying to create a general ideal co...
Name: recommendations, Length: 494, dtype: object

In [16]:
word_count_matrix, count_vect = nlp.create_wordcount_matrix(future_research, max_df=0.3, min_df=2, ngram=(1,3))

In [17]:
word_count_matrix

<609x566 sparse matrix of type '<class 'numpy.int64'>'
	with 3024 stored elements in Compressed Sparse Row format>

In [18]:
LDA = LatentDirichletAllocation(n_components=3, random_state=42)
LDA.fit(word_count_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [19]:
pyLDAvis.sklearn.prepare(LDA, word_count_matrix, count_vect)

/usr/local/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.052133 -0.130006       1        1  38.021747
0     -0.117288  0.100462       2        1  33.977685
2      0.169421  0.029544       3        1  28.000568, topic_info=    Category       Freq          Term      Total  loglift  logprob
78   Default  42.000000          data  42.000000  30.0000  30.0000
450  Default  26.000000       studies  26.000000  29.0000  29.0000
522  Default  53.000000     usability  53.000000  28.0000  28.0000
61   Default  24.000000    contextual  24.000000  27.0000  27.0000
98   Default  22.000000   diary study  22.000000  26.0000  26.0000
..       ...        ...           ...        ...      ...      ...
355   Topic3   8.237788  quantitative  36.422298  -0.2135  -4.6668
262   Topic3   8.229515        method  37.288356  -0.2380  -4.6678
342   Topic3   6.494803   qualitative  27.106091  -0.1558  -4.9045
88    Topic3   6.155583        design  33.064132  -0.4081  -4.9582
370   Topic3   5.537491        remote  30.508606  -0.4335  -5.0640

[182 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         2  0.941903       access
6         1  0.841569           ai
7         1  0.219099     analysis
7         2  0.350558     analysis
7         3  0.438198     analysis
...     ...       ...          ...
550       1  0.785485  ux research
552       2  0.941858           ve
553       3  0.951175        visit
561       2  0.841335         work
561       3  0.084133         work

[203 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [20]:
LDA.transform(word_count_matrix)

array([[0.04865894, 0.48496094, 0.46638011],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16682112, 0.66314436, 0.17003452],
       ...,
       [0.08755814, 0.82907687, 0.083365  ],
       [0.51182215, 0.45753248, 0.03064537],
       [0.02645341, 0.58808696, 0.38545963]])

In [21]:
nlp.find_top_documents_per_topic(LDA.transform(word_count_matrix), future_research, 3)

Top 3 Documents for Topic 0: 

Document 1
I'd love to just flat-out assemble a customer "council" database so that we can engage in more frequent and lighter-weight customer research sessions, a I've done at previous jobs, but there are major cultural hurdle - managing sales/channel partner relationship and mistrust, etc. This is still my major goal that I don't want to fully give up on, though.

Document 2
More contextual/observational studies, unmoderated studies, more co-design workshops, generally looking for opportunity to be more lean, quick, and agile with the process (leveraging existing insight from past research is a large part of that effort for us)

Document 3
more service design, design thinking learning proper SPSS quant method to complement qual method going to try out Delve for grounded theory analysis

Top 3 Documents for Topic 1: 

Document 1
User Experience Research (workshops, surveys, interviews, usability testing, benchmark, card sorting, etc.) and I would like to

In [22]:
from textblob import TextBlob, Word

In [46]:
word = 'studies'

In [33]:
w = Word(word)
w.lemmatize()

'workshop'

In [34]:
import nltk

In [35]:
wnl = nltk.stem.WordNetLemmatizer()

In [47]:
wnl.lemmatize(word)

'study'

In [ ]:
lemmas = [wnl.lemmatize(word) for word in text.split()]